In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Dataset
from tqdm import tqdm
from torch.utils.data import random_split
import torch.nn.functional as F


In [ ]:
torch.cuda.is_available()

In [ ]:
# !pip install kaggle -q
# !mkdir -p ~/.kaggle
# !mv kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json
# !pip install gdown -q
# !gdown 'https://drive.google.com/uc?id=16cdd1iYelIMSGELg56gD8y9vVP2GFAfu'
# !gdown https://drive.google.com/file/d/16cdd1iYelIMSGELg56gD8y9vVP2GFAfu/view?usp=drive_link


In [ ]:
# import gdown
# url = 'https://drive.google.com/uc?id=16cdd1iYelIMSGELg56gD8y9vVP2GFAfu'
# output = './data'
# gdown.download(url, output, quiet=False)


In [ ]:
# import zipfile
# zip_path = "./data/cse-251-b-2025.zip"
# extract_to = "./data/"
# with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_to)

In [ ]:
def getData(path):
    train_file = np.load(path+"/train.npz")
    train_data = train_file['data']
    test_file = np.load(path+"/test_input.npz")
    test_data = test_file['data']
    print(f"Training Data's shape is {train_data.shape} and Test Data's is {test_data.shape}")
    return train_data, test_data

In [ ]:
trainData, testData = getData("./data")
trainData.shape, testData.shape

In [ ]:
class WindowedNormalizedDataset(Dataset):
    def __init__(self, data, window_size, forecast_horizon, mean=None, std=None):
        self.data = data
        self.window_size = window_size
        self.forecast_horizon = forecast_horizon
        self.mean = mean
        self.std = std

        # Precompute indices of valid (sample, t) combinations
        self.indices = []
        for sample in range(data.shape[0]):
            for t in range(data.shape[2] - window_size - forecast_horizon + 1):
                self.indices.append((sample, t))

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        sample_idx, t = self.indices[idx]
        
        x = self.data[sample_idx, :, t:t+self.window_size, :]  # shape: (50, 40, 6)
        y = self.data[sample_idx, 0, t+self.window_size:t+self.window_size+self.forecast_horizon, :2]  # shape: (10, 2)

        if self.mean is not None and self.std is not None:
            x = (x - self.mean) / self.std
        
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)


# testdataset = WindowedNormalizedDataset(data = testData, window_size=50, forecast_horizon=20)
# len(dataset)

In [ ]:
torch.manual_seed(42)
dataset = WindowedNormalizedDataset(data = trainData, window_size=50, forecast_horizon=20)

# print(len(dataset), dataset)
# train_size = int(0.85 * len(dataset))
# test_size = len(dataset) - train_size
# train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
# # print(f"TrainDataset is {train_dataset[0]} and Test Dataset is {test_dataset[0]}")

In [ ]:
class EncoderDecoderModel(nn.Module):
    def __init__(self, infeatures, outfeatures = 2):
        super().__init__()
        self.layer1 = nn.Linear(in_features = infeatures, out_features = 16)
        self.layer2 = nn.Linear(in_features = 16, out_features = 32)
        self.layer3 = nn.Linear(in_features = 32, out_features = 64)
        self.encoderlstm = nn.LSTM(input_size = 64, hidden_size = 128, num_layers = 2, batch_first = True, dropout = 0.3)

        self.pool = nn.AdaptiveAvgPool1d(20)
        self.dropout = nn.Dropout(0.2)

        self.decoderlstm = nn.LSTM(input_size = 128, hidden_size = 64, num_layers = 2, batch_first = True, dropout = 0.3)
        self.layer10 = nn.Linear(in_features = 64, out_features = 32)
        self.layer11 = nn.Linear(in_features = 32, out_features = 16)
        self.layer12 = nn.Linear(in_features = 16, out_features = outfeatures)

        # layer 1 and layer 12 skip
        # layer 2 and layer 11 skip
        # layer 3 and layer 10 skip
        # encoder lstm to decoder lstm skip
        self.skip1 = nn.Linear(in_features = 16, out_features = 16)
        self.skip2 = nn.Linear(in_features = 32, out_features = 32)
        self.skip3 = nn.Linear(in_features = 64, out_features = 64)
        self.skip4 = nn.Linear(in_features = 128, out_features = 128)
        

    def forward(self, x):

        batch_size, channels, height, width = x.shape

        out1 = self.layer1(x)
        out1 = nn.ReLU()(out1)
        
        out2 = self.layer2(out1)
        out2 = nn.ReLU()(out2)
        
        out3 = self.layer3(out2)
        out3 = nn.ReLU()(out3)
        
        # print(x.shape, x.size(-1))
        tempout3 = out3.view(batch_size, -1, out3.size(-1))
        
        out4, _ = self.encoderlstm(tempout3)

        tempout4 = out4.permute(0, 2, 1)  # [batch, 64, seq_len]
       

        tempout4 = self.pool(tempout4)  # Forces output to [batch, channel, 10]

        tempout4 = tempout4.permute(0, 2, 1)  # [batch, 64, seq_len]

        lstmskip = tempout4 + self.skip4(tempout4)
        
        out5, _ = self.decoderlstm(lstmskip)
        
        out3_pooled = F.adaptive_avg_pool2d(out3.permute(0, 3, 1, 2), (20, 1))  # [128, 64, 20, 1]
        out3_reduced = out3_pooled.squeeze(-1).permute(0, 2, 1)  # [128, 20, 64]
        
        # print(out3_reduced.shape, out5.shape)
        mlpskip1 = out3_reduced + self.skip3(out5)
        # mlpskip1 = out3 + self.skip3(out5)
        out6 = self.layer10(mlpskip1)
        out6 = nn.ReLU()(out6)

        out2_pooled = F.adaptive_avg_pool2d(out2.permute(0, 3, 1, 2), (20, 1))  # [128, 64, 20, 1]
        out2_reduced = out2_pooled.squeeze(-1).permute(0, 2, 1)  # [128, 20, 64]

        mlpskip2 = out2_reduced + self.skip2(out6)
        out7 = self.layer11(mlpskip2)
        out7 = nn.ReLU()(out7)

        # print(out1.shape, out7.shape)
        out1_pooled = F.adaptive_avg_pool2d(out1.permute(0, 3, 1, 2), (20, 1))  # [128, 64, 20, 1]
        out1_reduced = out1_pooled.squeeze(-1).permute(0, 2, 1)  # [128, 20, 64]

        mlpskip3 = out1_reduced + self.skip1(out7)
        # print(mlpskip3.shape)
        out8 = self.layer12(mlpskip3)

        return out8

model = EncoderDecoderModel(6, 2)
# test = torch.randn(128, 50, 50, 6)
# out = model(test)
# out.shape
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")

In [ ]:
# trainDataset = LargeDataset(a, b, train_mean, train_std) # testing for small dataset a, b
torch.cuda.empty_cache()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.load_state_dict(torch.load("./models/modelD/medium_model_1017.472719.pth"))  



trainDataLoader = DataLoader(dataset, batch_size=128, shuffle=True, num_workers=0)
# testDataLoader = DataLoader(test_dataset, batch_size=128)
model.to(device)
# print(len(trainDataLoader), len(testDataLoader))
# Training setup
epochs = 100
lossFn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0008)

for each_epoch in range(epochs):
    model.train()
    runningLoss = 0.0
    loop = tqdm(trainDataLoader, desc=f"Epoch [{each_epoch+1}/{epochs}]")

    for batchX, batchY in loop:
        batchX, batchY = batchX.to(device, non_blocking=True), batchY.to(device, non_blocking=True)
        output = model(batchX)
        loss = lossFn(output, batchY)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        runningLoss += loss.item()

    avgLoss = runningLoss / len(trainDataLoader)

    # model.eval()
    # with torch.inference_mode():
    #     testloss = 0.0
    #     for testX, testY in testDataLoader:
    #         testX, testY = testX.to(device, non_blocking=True), testY.to(device, non_blocking=True)
    #         pred = model(testX)
    #         tloss = lossFn(pred, testY)
    #         testloss += tloss.item()

    #     avgtestloss = testloss/len(testDataLoader)
        # if each_epoch % 5 == 0 or each_epoch+1 >= epochs:
    print(f"Epoch {each_epoch + 1}, Training Loss: {avgLoss:.4f}")
    torch.save(model.state_dict(), f'./models/modelD/medium_model_{avgLoss:.6f}.pth')
    torch.cuda.empty_cache()
